In [1]:
#This notebook is implementing a tournament with spatial topology for the axelrod-python library. 
#Here will we only run small tournaments with deterministic strategies and will intersect the result using
#the simple tournament class for the axelrod library

In [2]:
#import the axelrod library
import axelrod as axl

In [2]:
# create some players 
players = (axl.Cooperator(), axl.Defector(), 
           axl.TitForTat(), axl.Alternator())

In [3]:
# run a simple tournament that everybody plays with everybody
tournament = axl.Tournament(players, turns=2, repetitions=2)
results = tournament.play()

In [4]:
#visualise the iteractions so we can compare to the spatial ones
for index_pair, interaction in results.interactions.items():
    player1 = tournament.players[index_pair[0]]
    player2 = tournament.players[index_pair[1]]
    print('%s vs %s: %s' % (player1, player2, interaction)) 

Cooperator vs Defector: [[('C', 'D'), ('C', 'D')], [('C', 'D'), ('C', 'D')]]
Defector vs Tit For Tat: [[('D', 'C'), ('D', 'D')], [('D', 'C'), ('D', 'D')]]
Cooperator vs Cooperator: [[('C', 'C'), ('C', 'C')], [('C', 'C'), ('C', 'C')]]
Tit For Tat vs Alternator: [[('C', 'C'), ('C', 'D')], [('C', 'C'), ('C', 'D')]]
Alternator vs Alternator: [[('C', 'C'), ('D', 'D')], [('C', 'C'), ('D', 'D')]]
Tit For Tat vs Tit For Tat: [[('C', 'C'), ('C', 'C')], [('C', 'C'), ('C', 'C')]]
Defector vs Alternator: [[('D', 'C'), ('D', 'D')], [('D', 'C'), ('D', 'D')]]
Cooperator vs Alternator: [[('C', 'C'), ('C', 'D')], [('C', 'C'), ('C', 'D')]]
Cooperator vs Tit For Tat: [[('C', 'C'), ('C', 'C')], [('C', 'C'), ('C', 'C')]]
Defector vs Defector: [[('D', 'D'), ('D', 'D')], [('D', 'D'), ('D', 'D')]]


In [5]:
#The first test
# create some edges 
edges = [(0, 1),(2, 3)]

In [6]:
#create a spatial match
sp=axl.SpatialMatches(players, turns=2, game = None , repetitions=2, edges=edges)

AttributeError: module 'axelrod' has no attribute 'SpatialMatches'

In [15]:
#looking at the chunk, to make sure everything was implement correct.
list(sp.build_match_chunks())

[((0, 1), (2, None, None, 0), 2),
 ((1, 2), (2, None, None, 0), 2),
 ((2, 3), (2, None, None, 0), 2),
 ((0, 3), (2, None, None, 0), 2)]

In [16]:
#create a spatial tournament, usuing the SpatialTournament class we implemented
spatial_tournament = axl.SpatialTournament(players, match_generator=sp ,turns=2, repetitions=2, edges=edges)

In [3]:
spatial_tournament = axl.SpatialTournament(players, match_generator=sp ,turns=2, repetitions=2, edges=edges)
spatial_results= spatial_tournament.play() 
for index_pair, interaction in spatial_results.interactions.items():
    player1 = spatial_tournament.players[index_pair[0]]
    player2 = spatial_tournament.players[index_pair[1]]
    print('%s vs %s: %s' % (player1, player2, interaction)) 

NameError: name 'players' is not defined

In [6]:
# Second Test : Playing themselves
edges = [(0, 0), (1, 1), (2, 2), (3, 3)]

In [21]:
# Third test : Value Error in case a player does not compete
edges = [(0, 1), (1, 2)]

In [23]:
# Fourth test
edges = [(0, 1), (2, 3)]

In [24]:
#here we create the match class, the tournament class and visualise the results
sp=axl.SpatialMatches(players, turns=2, game = None , repetitions=2, edges=edges)
spatial_tournament = axl.SpatialTournament(players, match_generator=sp ,turns=2, repetitions=2, edges=edges)
spatial_results= spatial_tournament.play() 
for index_pair, interaction in spatial_results.interactions.items():
    player1 = spatial_tournament.players[index_pair[0]]
    player2 = spatial_tournament.players[index_pair[1]]
    print('%s vs %s: %s' % (player1, player2, interaction)) 

Cooperator vs Defector: [[('C', 'D'), ('C', 'D')], [('C', 'D'), ('C', 'D')]]
Tit For Tat vs Alternator: [[('C', 'C'), ('C', 'D')], [('C', 'C'), ('C', 'D')]]
